In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

#expands all columns in datafrae
pd.options.display.max_columns = None

#gets rid of SettingWithCopyWarning:
pd.options.mode.chained_assignment = None

In [2]:
#import the data
data = pd.read_json("data/data.json")

In [3]:
data.head(2)

,acct_type,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,event_end,event_published,event_start,fb_published,gts,has_analytics,has_header,has_logo,listed,name,name_length,num_order,num_payouts,object_id,org_desc,org_facebook,org_name,org_twitter,payee_name,payout_type,previous_payouts,sale_duration,sale_duration2,show_map,ticket_types,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state
0,fraudster_event,1266062400,3852,5,US,USD,0.0,"<p><a href=""http://s432.photobucket.com/albums...",gmail.com,1262739706,1265630400,1.263110e+09,1265594400,0,0.00,0,1.0,0,y,"99 HOUR ""NO SLEEP"" SUPER BOWL CELEBRITY WEEKEN...",60,0,0,527017,,0.0,Party Starz Ent & Diverse Int'l Group,0.0,,,[],29.0,33,1,"[{'event_id': 527017, 'cost': 25.0, 'availabil...",36,1259613950,1,717 Washington Avenue,US,25.777471,-80.133433,INK Nightclub - South Beach,FL
1,premium,1296720000,3499,0,US,USD,1.0,"<p>Join us for a quick, one-night, community-b...",ruf.org,1293832670,1296288000,1.293833e+09,1296255600,0,868.02,0,0.0,1,n,Winthrop RUF Winter Getaway,27,23,1,786878,"<p>Since 1987, RUF has ministered to students ...",0.0,RUF at Winthrop University,12.0,RUF,CHECK,"[{'name': 'RUF', 'created': '2010-10-01 01:10:...",28.0,28,0,"[{'event_id': 786878, 'cost': 35.0, 'availabil...",149,1280942776,3,,US,32.776566,-79.930922,"The Charleston, SC area",SC


In [4]:
#number of features
print ("There are {} features.".format(data.shape[1]))

There are 44 features.


In [5]:
#number of datapoints
print ("there are {} datapoints.".format(data.shape[0]))

there are 14337 datapoints.


### Feature Engineering

##### ACH or Check?

In [6]:
data.payout_type.unique()

array(['', 'CHECK', 'ACH'], dtype=object)

In [7]:
payout_type = pd.get_dummies(data.payout_type)

In [8]:
data = data.join(payout_type)

In [9]:
data.sample(2)

,acct_type,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,event_end,event_published,event_start,fb_published,gts,has_analytics,has_header,has_logo,listed,name,name_length,num_order,num_payouts,object_id,org_desc,org_facebook,org_name,org_twitter,payee_name,payout_type,previous_payouts,sale_duration,sale_duration2,show_map,ticket_types,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state,,ACH,CHECK
11532,premium,1306193400,5910,8,RO,EUR,1.0,"<h1 class=""securitytraining_image"">Security Tr...",ecsl.eu,1300715303,1305761400,1.300715e+09,1305649800,0,214.29,0,0.0,1,y,Security Training 17th & 18th May 2011,38,1,2,1463183,<p>Kingstown College in association with Exec...,16.0,Kingstown College,12.0,,ACH,"[{'name': '', 'created': '2011-04-19 01:14:48'...",57.0,57,1,"[{'event_id': 1463183, 'cost': 140.25, 'availa...",91,1292847897,3,IDA Business Park,IE,53.406188,-6.444369,Ballycoolin Regional Training Centre,Dublin,0,1,0
5080,premium,1323792000,14341,0,GB,EUR,1.0,"<p style=""color: #000000;"">EPEMED and The Luxe...",collegehill.com,1314710807,1323360000,1.314713e+09,1323306000,0,28402.11,0,1.0,1,n,Personalised Medicine in Europe:<br/> What wil...,67,94,0,2107133,"<p>EPEMED website <a href=""http://www.epemed.o...",0.0,EPEMED and The Luxembourg Life Sciences Initia...,0.0,,ACH,"[{'name': '', 'created': '2011-12-13 03:11:58'...",99.0,99,0,"[{'event_id': 2107133, 'cost': 140.25, 'availa...",42,1311085071,1,6 rue Fort Niedergrünewald,LU,0.000000,0.000000,Hotel Novotel Luxembourg-Kirchberg,Luxembourg,0,1,0


##### description feature

In [10]:
#what's in the decription for the first event?
data.description.iloc[0]

'<p><a href="http://s432.photobucket.com/albums/qq49/digusonline/?action=view&amp;current=supersunday.jpg" target="_blank"><img src="http://i432.photobucket.com/albums/qq49/digusonline/supersunday.jpg" border="0" alt="Photobucket" /></a></p>\r\n<p>\xa0</p>\r\n<p style="text-align: center;"><font size="3"><strong>Party Starz Entertaintment &amp; Diverse International Group Presents...<br /></strong></font><br /><font face="tahoma,arial,helvetica,sans-serif" size="4"><strong>The Official\xa0"99 Hour No Sleep" Super Bowl </strong></font></p>\r\n<p style="text-align: center;"><font face="tahoma,arial,helvetica,sans-serif" size="4"><strong>Weekend </strong></font><font face="tahoma,arial,helvetica,sans-serif" size="4"><strong>Grand Finale</strong></font></p>\r\n<p><br /><span style="font-family: Times New Roman,serif;"><font size="3"><font face="tahoma,arial,helvetica,sans-serif">No matter who wins or loses, this post-game party cannot be missed! Enjoy the drink specials and all night music

In [11]:
from bs4 import BeautifulSoup
import re

In [12]:
#Parse the HTML tags
soup = BeautifulSoup(data.description.iloc[0]).text.replace("\xa0", " ").replace("\n", " ").lower()

In [13]:
soup

'   party starz entertaintment & diverse international group presents...the official "99 hour no sleep" super bowl  weekend grand finale no matter who wins or loses, this post-game party cannot be missed! enjoy the drink specials and all night music! the ultimate "99 no sleep" super bowl weekend grand finale! sunday, february 7th, 2010 @ ink nightclub-south beach (717 washington ave, miami bch, fl 33139). hosted by 99 jamz with special appearances by fast life youngstaz. special invited guests: billy blue, dorrough and brisco and with performances by various artist\'s and many many more at south beach miami\'s finest venue! this is a party you have to see to believe! this is everyone\'s last chance to make that impression, so don\'t half-step, bring your "a" game and party with all of the hottest artists, celebrities, models and superstar athletes that are guaranteed to be in the building! dj q & dj slym will ber keepig the party jumping til the sun comes up with the hottest hip-hop, r

In [14]:
import string
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer

In [15]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/michellehoang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/michellehoang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/michellehoang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
#tokenize and lose punctuation

In [17]:
import string

In [18]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [19]:
soup = soup.translate(str.maketrans('', '', string.punctuation))

In [20]:
soup

'   party starz entertaintment  diverse international group presentsthe official 99 hour no sleep super bowl  weekend grand finale no matter who wins or loses this postgame party cannot be missed enjoy the drink specials and all night music the ultimate 99 no sleep super bowl weekend grand finale sunday february 7th 2010  ink nightclubsouth beach 717 washington ave miami bch fl 33139 hosted by 99 jamz with special appearances by fast life youngstaz special invited guests billy blue dorrough and brisco and with performances by various artists and many many more at south beach miamis finest venue this is a party you have to see to believe this is everyones last chance to make that impression so dont halfstep bring your a game and party with all of the hottest artists celebrities models and superstar athletes that are guaranteed to be in the building dj q  dj slym will ber keepig the party jumping til the sun comes up with the hottest hiphop rb reggae old school and more  doors open  10pm

In [21]:
data['parsed_desc'] = 1
for n in range(data.shape[0]):
    soup = BeautifulSoup(data.description.iloc[n]).text.replace("\xa0", " ").replace("\n", " ").lower()
    data["parsed_desc"].iloc[n] = soup.translate(str.maketrans('','',string.punctuation))

In [22]:
data.head(2)

,acct_type,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,event_end,event_published,event_start,fb_published,gts,has_analytics,has_header,has_logo,listed,name,name_length,num_order,num_payouts,object_id,org_desc,org_facebook,org_name,org_twitter,payee_name,payout_type,previous_payouts,sale_duration,sale_duration2,show_map,ticket_types,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state,,ACH,CHECK,parsed_desc
0,fraudster_event,1266062400,3852,5,US,USD,0.0,"<p><a href=""http://s432.photobucket.com/albums...",gmail.com,1262739706,1265630400,1.263110e+09,1265594400,0,0.00,0,1.0,0,y,"99 HOUR ""NO SLEEP"" SUPER BOWL CELEBRITY WEEKEN...",60,0,0,527017,,0.0,Party Starz Ent & Diverse Int'l Group,0.0,,,[],29.0,33,1,"[{'event_id': 527017, 'cost': 25.0, 'availabil...",36,1259613950,1,717 Washington Avenue,US,25.777471,-80.133433,INK Nightclub - South Beach,FL,1,0,0,party starz entertaintment diverse interna...
1,premium,1296720000,3499,0,US,USD,1.0,"<p>Join us for a quick, one-night, community-b...",ruf.org,1293832670,1296288000,1.293833e+09,1296255600,0,868.02,0,0.0,1,n,Winthrop RUF Winter Getaway,27,23,1,786878,"<p>Since 1987, RUF has ministered to students ...",0.0,RUF at Winthrop University,12.0,RUF,CHECK,"[{'name': 'RUF', 'created': '2010-10-01 01:10:...",28.0,28,0,"[{'event_id': 786878, 'cost': 35.0, 'availabil...",149,1280942776,3,,US,32.776566,-79.930922,"The Charleston, SC area",SC,0,0,1,join us for a quick onenight communitybuilding...


In [23]:
#stop words
stop_words = set(nltk.corpus.stopwords.words('english'))

In [ ]:
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [ ]:
len(data.parsed_desc.iloc[0])

1406

In [ ]:
desc_no_stops = []
for word in data.parsed_desc.iloc[0].split():
    if word not in stop_words:
        desc_no_stops.append(word)

In [ ]:
desc_no_stops

['party',
 'starz',
 'entertaintment',
 'diverse',
 'international',
 'group',
 'presentsthe',
 'official',
 '99',
 'hour',
 'sleep',
 'super',
 'bowl',
 'weekend',
 'grand',
 'finale',
 'matter',
 'wins',
 'loses',
 'postgame',
 'party',
 'cannot',
 'missed',
 'enjoy',
 'drink',
 'specials',
 'night',
 'music',
 'ultimate',
 '99',
 'sleep',
 'super',
 'bowl',
 'weekend',
 'grand',
 'finale',
 'sunday',
 'february',
 '7th',
 '2010',
 'ink',
 'nightclubsouth',
 'beach',
 '717',
 'washington',
 'ave',
 'miami',
 'bch',
 'fl',
 '33139',
 'hosted',
 '99',
 'jamz',
 'special',
 'appearances',
 'fast',
 'life',
 'youngstaz',
 'special',
 'invited',
 'guests',
 'billy',
 'blue',
 'dorrough',
 'brisco',
 'performances',
 'various',
 'artists',
 'many',
 'many',
 'south',
 'beach',
 'miamis',
 'finest',
 'venue',
 'party',
 'see',
 'believe',
 'everyones',
 'last',
 'chance',
 'make',
 'impression',
 'dont',
 'halfstep',
 'bring',
 'game',
 'party',
 'hottest',
 'artists',
 'celebrities',
 'mod

In [ ]:
data["parsed_desc_no_stops"] = ""
for i in range(data.shape[0]):
    desc_no_stops = []
    for word in data.parsed_desc.iloc[i].split():
        if word not in stop_words:
            desc_no_stops.append(word)
    
    data.parsed_desc_no_stops.iloc[i] = ' '.join(desc_no_stops)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [ ]:
corpus = data.parsed_desc_no_stops.tolist()

In [ ]:
corpus

['party starz entertaintment diverse international group presentsthe official 99 hour sleep super bowl weekend grand finale matter wins loses postgame party cannot missed enjoy drink specials night music ultimate 99 sleep super bowl weekend grand finale sunday february 7th 2010 ink nightclubsouth beach 717 washington ave miami bch fl 33139 hosted 99 jamz special appearances fast life youngstaz special invited guests billy blue dorrough brisco performances various artists many many south beach miamis finest venue party see believe everyones last chance make impression dont halfstep bring game party hottest artists celebrities models superstar athletes guaranteed building dj q dj slym ber keepig party jumping til sun comes hottest hiphop rb reggae old school doors open 10pm ladies 18 21 gents 241 drinks 1am plus best bottle specials night long offering super bowl game ticket stub discounts night dress code enforced wear wear allnight drink specials information table reservations email 99

In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

In [ ]:
print(vectorizer.get_feature_names())

['00', '000', '0000', '000000', '0000043', '00001194', '00001195', '00001198', '0006', '0007', '000em', '0010mobile', '0017029392525', '0020', '003', '0031', '00320', '00334', '005', '00500', '00557', '007', '0074', '007pm', '007themed', '0084166', '008in', '0090005140', '0099', '00h30', '00px', '00s', '01', '0100', '0100pm', '010120125', '0102', '01030pm', '01072011', '01072013', '0108', '010emfontfamily', '011', '01105', '01122012', '0113', '0114', '0115', '011513', '01159932350', '0116', '0117', '01200am', '012013', '01202', '0121', '01223', '0123', '01245', '0125', '01254', '01270', '01273', '01274', '01275', '013', '0130', '01302013', '0130pm', '0131', '0135', '01352', '0138', '014', '01400', '0141', '01416', '01425', '0145', '01452', '01462', '01473', '01480', '01482', '015', '01509', '01519091', '01522', '0154', '01543', '01603', '01604', '0161', '016251135', '01633', '01639', '01642', '017', '01729', '01760', '01761', '01763', '01765', '01772', '018', '0180', '0184', '01841', '

In [ ]:
cv_array = X.toarray()

In [ ]:
count_vectorizer = pd.DataFrame(cv_array, columns = vectorizer.get_feature_names())

In [ ]:
count_vectorizer

##### Convert epoch time to date time

In [ ]:
from datetime import datetime

In [ ]:
#change all epoch elements to dates
data["date_created"] = pd.to_datetime(data.event_created, unit='s').dt.date
data["date_published"] = pd.to_datetime(data.event_published, unit='s').dt.date
data["start"] = pd.to_datetime(data.event_start, unit='s').dt.date
data["end"] = pd.to_datetime(data.event_end, unit='s').dt.date

In [ ]:
data.sample(2)

### Create Label

In [ ]:
data.acct_type.unique()

In [ ]:
#add a new column called "Fraud"
#True if any acct_type field contains the word fraud
#False if it does not contain it
data["Fraud"] = data.acct_type.str.contains("fraud", regex = True)

In [ ]:
data.head(2)

In [ ]:
#this will be our target
label = data["Fraud"]

## Split our data into train and test data

In [ ]:
X = data.drop("Fraud", axis = 1)
y = data["Fraud"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33)

In [ ]:
events = X_train
actual = y_train

## Start EDA

In [ ]:
events.describe()

In [ ]:
events.info()

In [ ]:
#checking to see if any features of NaN values
#true indicates the feature has NaN values
events.isna().any()

##### body_length

In [ ]:
events.body_length.value_counts()

##### currency types

In [ ]:
#what currencies were used?
currencies = events.currency.unique()
currencies